# 🧠 Deep Learning Model V2 — Improved CNN + BiLSTM + Attention
### Key improvements over V1:
1. **Multi-scale CNN** — parallel Conv1d with kernels 3, 7, 13 to capture hourly + daily + weekly patterns
2. **Deeper BiLSTM** — 2-layer stacked BiLSTM with residual connections
3. **Temporal Decoder** — learned upsampling from compressed representation instead of global avg pool → single linear
4. **Peak-Aware Loss** — Huber + weighted penalty on top-K load hours
5. **Stronger Regularization** — spatial dropout, weight decay, mixup augmentation
6. **Cosine Annealing with Warmup** — better LR schedule than ReduceLROnPlateau
7. **Gradient accumulation** — effective batch size 256 for stable training


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, time, math, numpy as np, joblib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings; warnings.filterwarnings('ignore')

# Custom JSON encoder for numpy types
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer,)):
            return int(obj)
        if isinstance(obj, (np.floating,)):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

BASE_DIR  = '/content/drive/MyDrive/Electricity_Load_Forecast'
MODEL_DIR = os.path.join(BASE_DIR, 'models')
V2_DIR    = os.path.join(MODEL_DIR, 'v2')
os.makedirs(V2_DIR, exist_ok=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
config         = joblib.load(os.path.join(MODEL_DIR, 'config.pkl'))
target_scaler  = joblib.load(os.path.join(MODEL_DIR, 'target_scaler.pkl'))

X_train_w = np.load(os.path.join(MODEL_DIR, 'X_train_w.npy'))
y_train_w = np.load(os.path.join(MODEL_DIR, 'y_train_w.npy'))
X_val_w   = np.load(os.path.join(MODEL_DIR, 'X_val_w.npy'))
y_val_w   = np.load(os.path.join(MODEL_DIR, 'y_val_w.npy'))
X_test_w  = np.load(os.path.join(MODEL_DIR, 'X_test_w.npy'))
y_test_w  = np.load(os.path.join(MODEL_DIR, 'y_test_w.npy'))

N_FEATURES = config['N_FEATURES']
INPUT_LEN  = config['INPUT_LEN']
OUTPUT_LEN = config['OUTPUT_LEN']
print(f"Features: {N_FEATURES}, Input: {INPUT_LEN}h, Output: {OUTPUT_LEN}h")
print(f"Train: {X_train_w.shape}, Val: {X_val_w.shape}, Test: {X_test_w.shape}")

In [ ]:
HP = {
    'batch_size':      64,
    'accum_steps':     4,
    'lr':              3e-4,
    'weight_decay':    1e-3,
    'max_epochs':      80,
    'patience':        20,
    'warmup_epochs':   5,
    'grad_clip':       0.5,
    'dropout':         0.2,
    'spatial_dropout':  0.1,
    'noise_std':       0.02,
    'mixup_alpha':     0.2,
    'conv_filters':    64,
    'lstm_hidden':     192,
    'lstm_layers':     2,
    'n_heads':         8,
    'decoder_hidden':  256,
    'huber_delta':     1.0,
    'peak_weight':     2.0,
    'peak_fraction':   0.2,
}
print("Hyperparameters:", json.dumps(HP, indent=2))

In [ ]:
def make_loader(X, y, batch_size, shuffle=True):
    ds = TensorDataset(torch.tensor(X), torch.tensor(y))
    return DataLoader(ds, batch_size=batch_size, shuffle=shuffle,
                      num_workers=2, pin_memory=True, drop_last=shuffle)

train_loader = make_loader(X_train_w, y_train_w, HP['batch_size'], shuffle=True)
val_loader   = make_loader(X_val_w,   y_val_w,   HP['batch_size'], shuffle=False)
test_loader  = make_loader(X_test_w,  y_test_w,  HP['batch_size'], shuffle=False)
print(f"Train batches: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}")

In [ ]:
# ═══════════════════════════════════════
#  IMPROVED MODEL ARCHITECTURE V2
# ═══════════════════════════════════════

class MultiScaleCNN(nn.Module):
    """Parallel Conv1d with different kernel sizes to capture multi-scale patterns."""
    def __init__(self, in_channels, out_channels, dropout=0.1):
        super().__init__()
        self.conv3  = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.conv7  = nn.Conv1d(in_channels, out_channels, kernel_size=7, padding=3)
        self.conv13 = nn.Conv1d(in_channels, out_channels, kernel_size=13, padding=6)
        self.bn     = nn.BatchNorm1d(out_channels * 3)
        self.proj   = nn.Conv1d(out_channels * 3, out_channels, kernel_size=1)
        self.bn2    = nn.BatchNorm1d(out_channels)
        self.drop   = nn.Dropout(dropout)

    def forward(self, x):
        c3  = F.gelu(self.conv3(x))
        c7  = F.gelu(self.conv7(x))
        c13 = F.gelu(self.conv13(x))
        out = torch.cat([c3, c7, c13], dim=1)
        out = F.gelu(self.bn(out))
        out = self.drop(F.gelu(self.bn2(self.proj(out))))
        return out


class SpatialDropout1d(nn.Module):
    def __init__(self, p=0.1):
        super().__init__()
        self.p = p

    def forward(self, x):
        if not self.training or self.p == 0:
            return x
        mask = torch.bernoulli(torch.ones(x.shape[0], 1, x.shape[2], device=x.device) * (1 - self.p))
        return x * mask / (1 - self.p)


class TemporalDecoder(nn.Module):
    """Learned temporal queries + cross-attention decoder."""
    def __init__(self, d_model, pred_len, n_heads=8, dropout=0.2):
        super().__init__()
        self.pred_len = pred_len
        self.query_embed = nn.Parameter(torch.randn(1, pred_len, d_model) * 0.02)
        self.cross_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.norm1 = nn.LayerNorm(d_model)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 2), nn.GELU(), nn.Dropout(dropout),
            nn.Linear(d_model * 2, d_model), nn.Dropout(dropout)
        )
        self.norm2 = nn.LayerNorm(d_model)
        self.out_proj = nn.Linear(d_model, 1)

    def forward(self, encoder_out):
        B = encoder_out.shape[0]
        queries = self.query_embed.expand(B, -1, -1)
        attn_out, _ = self.cross_attn(queries, encoder_out, encoder_out)
        x = self.norm1(queries + attn_out)
        x = self.norm2(x + self.ffn(x))
        x = self.out_proj(x).squeeze(-1)
        return x


class LoadForecastV2(nn.Module):
    def __init__(self, n_features, seq_len, pred_len,
                 conv_filters=64, lstm_hidden=192, lstm_layers=2,
                 n_heads=8, decoder_hidden=256,
                 dropout=0.2, spatial_drop=0.1, noise_std=0.02):
        super().__init__()
        self.noise_std = noise_std
        self.input_proj = nn.Linear(n_features, conv_filters)
        self.spatial_drop = SpatialDropout1d(spatial_drop)
        self.ms_cnn = MultiScaleCNN(conv_filters, conv_filters, dropout=dropout)
        self.conv_block2 = nn.Sequential(
            nn.Conv1d(conv_filters, conv_filters, kernel_size=5, padding=2),
            nn.BatchNorm1d(conv_filters), nn.GELU(), nn.Dropout(dropout)
        )
        d_model = lstm_hidden * 2
        self.bilstm1 = nn.LSTM(conv_filters, lstm_hidden, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(d_model, lstm_hidden, batch_first=True, bidirectional=True)
        self.lstm_norm1 = nn.LayerNorm(d_model)
        self.lstm_norm2 = nn.LayerNorm(d_model)
        self.lstm_drop  = nn.Dropout(dropout)
        self.self_attn = nn.MultiheadAttention(d_model, n_heads, dropout=dropout, batch_first=True)
        self.attn_norm = nn.LayerNorm(d_model)
        self.attn_ffn  = nn.Sequential(
            nn.Linear(d_model, d_model * 2), nn.GELU(),
            nn.Dropout(dropout), nn.Linear(d_model * 2, d_model)
        )
        self.attn_ffn_norm = nn.LayerNorm(d_model)
        self.decoder = TemporalDecoder(d_model, pred_len, n_heads, dropout)

    def forward(self, x):
        if self.training and self.noise_std > 0:
            x = x + torch.randn_like(x) * self.noise_std
        x = self.input_proj(x)
        x = self.spatial_drop(x)
        x = x.permute(0, 2, 1)
        x = self.ms_cnn(x)
        x = x + self.conv_block2(x)
        x = x.permute(0, 2, 1)
        h1, _ = self.bilstm1(x)
        h1 = self.lstm_norm1(h1)
        h1 = self.lstm_drop(h1)
        h2, _ = self.bilstm2(h1)
        h2 = self.lstm_norm2(h1 + h2)
        attn_out, _ = self.self_attn(h2, h2, h2)
        h2 = self.attn_norm(h2 + attn_out)
        h2 = self.attn_ffn_norm(h2 + self.attn_ffn(h2))
        out = self.decoder(h2)
        return out


model = LoadForecastV2(
    n_features=N_FEATURES, seq_len=INPUT_LEN, pred_len=OUTPUT_LEN,
    conv_filters=HP['conv_filters'], lstm_hidden=HP['lstm_hidden'],
    lstm_layers=HP['lstm_layers'], n_heads=HP['n_heads'],
    decoder_hidden=HP['decoder_hidden'],
    dropout=HP['dropout'], spatial_drop=HP['spatial_dropout'],
    noise_std=HP['noise_std']
).to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable    = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total params: {total_params:,} | Trainable: {trainable:,}")
print(model)

In [ ]:
# ═══════════════════════════════════════
#  PEAK-AWARE LOSS + UTILITIES
# ═══════════════════════════════════════
class PeakAwareLoss(nn.Module):
    def __init__(self, delta=1.0, peak_weight=2.0, peak_fraction=0.2):
        super().__init__()
        self.huber = nn.HuberLoss(delta=delta, reduction='none')
        self.peak_weight = peak_weight
        self.peak_fraction = peak_fraction

    def forward(self, pred, target):
        base_loss = self.huber(pred, target)
        k = max(1, int(target.shape[1] * self.peak_fraction))
        topk_vals, _ = torch.topk(target, k, dim=1)
        threshold = topk_vals[:, -1:].detach()
        is_peak = (target >= threshold).float()
        weights = 1.0 + (self.peak_weight - 1.0) * is_peak
        return (base_loss * weights).mean()


def mixup_data(x, y, alpha=0.2):
    if alpha <= 0:
        return x, y
    lam = np.random.beta(alpha, alpha)
    lam = max(lam, 1 - lam)
    idx = torch.randperm(x.size(0), device=x.device)
    return lam * x + (1 - lam) * x[idx], lam * y + (1 - lam) * y[idx]


def cosine_warmup_scheduler(optimizer, warmup_epochs, max_epochs, steps_per_epoch):
    warmup_steps = warmup_epochs * steps_per_epoch
    total_steps  = max_epochs * steps_per_epoch
    def lr_lambda(step):
        if step < warmup_steps:
            return step / max(1, warmup_steps)
        progress = (step - warmup_steps) / max(1, total_steps - warmup_steps)
        return 0.5 * (1 + math.cos(math.pi * progress))
    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

print("✅ Loss, mixup, and scheduler defined")

In [ ]:
# ═══════════════════════════════════════
#  TRAINING LOOP V2
# ═══════════════════════════════════════
criterion = PeakAwareLoss(delta=HP['huber_delta'], peak_weight=HP['peak_weight'],
                          peak_fraction=HP['peak_fraction'])
optimizer = torch.optim.AdamW(model.parameters(), lr=HP['lr'], weight_decay=HP['weight_decay'])
scheduler = cosine_warmup_scheduler(optimizer, HP['warmup_epochs'], HP['max_epochs'], len(train_loader))

history = {'train_loss': [], 'val_loss': [], 'lr': []}
best_val_loss = float('inf')
patience_counter = 0
best_model_path = os.path.join(V2_DIR, 'best_dl_model_v2.pt')
ACCUM = HP['accum_steps']

print(f"\n{'='*60}")
print(f"Training V2 for up to {HP['max_epochs']} epochs (effective BS={HP['batch_size']*ACCUM})")
print(f"{'='*60}")

for epoch in range(HP['max_epochs']):
    t0 = time.time()
    model.train()
    train_losses = []
    optimizer.zero_grad()
    for step, (X_b, y_b) in enumerate(train_loader):
        X_b, y_b = X_b.to(device), y_b.to(device)
        X_b, y_b = mixup_data(X_b, y_b, HP['mixup_alpha'])
        pred = model(X_b)
        loss = criterion(pred, y_b) / ACCUM
        loss.backward()
        if (step + 1) % ACCUM == 0 or (step + 1) == len(train_loader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), HP['grad_clip'])
            optimizer.step()
            optimizer.zero_grad()
        scheduler.step()
        train_losses.append(loss.item() * ACCUM)
    train_loss = np.mean(train_losses)

    model.eval()
    val_losses = []
    with torch.no_grad():
        for X_b, y_b in val_loader:
            X_b, y_b = X_b.to(device), y_b.to(device)
            val_losses.append(criterion(model(X_b), y_b).item())
    val_loss = np.mean(val_losses)

    current_lr = optimizer.param_groups[0]['lr']
    history['train_loss'].append(float(train_loss))
    history['val_loss'].append(float(val_loss))
    history['lr'].append(float(current_lr))

    elapsed = time.time() - t0
    tag = ''
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        tag = ' ✅ BEST'
    else:
        patience_counter += 1

    print(f"Epoch {epoch+1:3d}/{HP['max_epochs']} | "
          f"Train: {train_loss:.6f} | Val: {val_loss:.6f} | "
          f"LR: {current_lr:.2e} | {elapsed:.1f}s{tag}")

    if patience_counter >= HP['patience']:
        print(f"\n⛔ Early stopping at epoch {epoch+1} (best val: {best_val_loss:.6f})")
        break

model.load_state_dict(torch.load(best_model_path, weights_only=True))
print(f"✅ Best V2 model loaded (val loss: {best_val_loss:.6f})")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
ax1.plot(history['train_loss'], label='Train Loss', color='#2196F3')
ax1.plot(history['val_loss'], label='Val Loss', color='#FF5722')
ax1.set_xlabel('Epoch'); ax1.set_ylabel('Peak-Aware Huber Loss')
ax1.set_title('Training & Validation Loss (V2)'); ax1.legend(); ax1.grid(True, alpha=0.3)
ax2.plot(history['lr'], color='green')
ax2.set_xlabel('Epoch'); ax2.set_ylabel('Learning Rate')
ax2.set_title('Cosine Warmup Schedule'); ax2.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(V2_DIR, 'dl_training_history_v2.png'), dpi=150)
plt.show()

In [ ]:
# ═══════════════════════════════════════
#  TEST EVALUATION
# ═══════════════════════════════════════
def predict_all(model, loader):
    model.eval()
    preds = []
    with torch.no_grad():
        for X_b, _ in loader:
            preds.append(model(X_b.to(device)).cpu().numpy())
    return np.concatenate(preds, axis=0)

def inverse_scale(y):
    return target_scaler.inverse_transform(y.reshape(-1, 1)).reshape(y.shape)

def compute_metrics(y_true, y_pred):
    yt, yp = inverse_scale(y_true), inverse_scale(y_pred)
    mae  = float(mean_absolute_error(yt.flatten(), yp.flatten()))
    rmse = float(np.sqrt(mean_squared_error(yt.flatten(), yp.flatten())))
    mask = yt.flatten() != 0
    mape = float(np.mean(np.abs((yt.flatten()[mask] - yp.flatten()[mask]) / yt.flatten()[mask])) * 100)
    peak_errors = [float(np.abs(yt[i, np.argmax(yt[i])] - yp[i, np.argmax(yt[i])])) for i in range(len(yt))]
    peak_mae = float(np.mean(peak_errors))
    return {'MAE': round(mae, 2), 'RMSE': round(rmse, 2),
            'MAPE': round(mape, 2), 'Peak_MAE': round(peak_mae, 2)}

y_pred_test = predict_all(model, test_loader)
y_pred_val  = predict_all(model, val_loader)

dl_v2_val  = compute_metrics(y_val_w[:len(y_pred_val)],   y_pred_val)
dl_v2_test = compute_metrics(y_test_w[:len(y_pred_test)], y_pred_test)

print(f"\n{'='*40}\nDL V2 Model — Validation\n{'='*40}")
for k, v in dl_v2_val.items():  print(f"  {k:10s}: {v}")
print(f"\n{'='*40}\nDL V2 Model — Test\n{'='*40}")
for k, v in dl_v2_test.items(): print(f"  {k:10s}: {v}")

In [ ]:
horizons = [1, 24, 72, 168]
yt_inv = inverse_scale(y_test_w[:len(y_pred_test)])
yp_inv = inverse_scale(y_pred_test)

horizon_metrics = {}
for h in horizons:
    idx = h - 1
    mae_h  = float(mean_absolute_error(yt_inv[:, idx], yp_inv[:, idx]))
    rmse_h = float(np.sqrt(mean_squared_error(yt_inv[:, idx], yp_inv[:, idx])))
    mask = yt_inv[:, idx] != 0
    mape_h = float(np.mean(np.abs((yt_inv[:, idx][mask] - yp_inv[:, idx][mask]) / yt_inv[:, idx][mask])) * 100)
    horizon_metrics[h] = {'MAE': round(mae_h, 2), 'RMSE': round(rmse_h, 2), 'MAPE': round(mape_h, 2)}

print(f"\n{'='*50}\nHorizon-Wise Error V2 (Test Set)\n{'='*50}")
print(f"{'Hour':>6} {'MAE':>10} {'RMSE':>10} {'MAPE%':>10}")
for h, m in horizon_metrics.items():
    print(f"{h:>6} {m['MAE']:>10} {m['RMSE']:>10} {m['MAPE']:>10}")

mae_per_hour = [float(mean_absolute_error(yt_inv[:, h], yp_inv[:, h])) for h in range(OUTPUT_LEN)]
plt.figure(figsize=(12, 5))
plt.plot(range(1, OUTPUT_LEN+1), mae_per_hour, 'b-', lw=1.5)
for h in horizons:
    plt.axvline(h, color='red', ls='--', alpha=0.5)
    plt.annotate(f'h={h}', (h, mae_per_hour[h-1]), fontsize=9, color='red')
plt.xlabel('Forecast Horizon (hours)'); plt.ylabel('MAE (MW)')
plt.title('Horizon-Wise MAE — DL V2 (Test Set)'); plt.grid(True, alpha=0.3)
plt.savefig(os.path.join(V2_DIR, 'horizon_wise_error_v2.png'), dpi=150)
plt.show()

In [ ]:
n_origins = 10
step = max(1, len(y_pred_test) // n_origins)
origin_indices = list(range(0, len(y_pred_test), step))[:n_origins]

rolling_metrics = []
for idx in origin_indices:
    X_single = torch.tensor(X_test_w[idx:idx+1]).to(device)
    with torch.no_grad():
        pred = model(X_single).cpu().numpy()
    m = compute_metrics(y_test_w[idx:idx+1], pred)
    m['origin_idx'] = int(idx)
    rolling_metrics.append(m)

print(f"\n{'='*60}\nRolling Origin Evaluation V2 ({n_origins} origins)\n{'='*60}")
print(f"{'Origin':>8} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
for rm in rolling_metrics:
    print(f"{rm['origin_idx']:>8} {rm['MAE']:>8} {rm['RMSE']:>8} {rm['MAPE']:>8} {rm['Peak_MAE']:>10}")

avg_rolling = {k: round(float(np.mean([m[k] for m in rolling_metrics])), 2)
               for k in ['MAE','RMSE','MAPE','Peak_MAE']}
print(f"{'AVG':>8} {avg_rolling['MAE']:>8} {avg_rolling['RMSE']:>8} "
      f"{avg_rolling['MAPE']:>8} {avg_rolling['Peak_MAE']:>10}")

In [ ]:
# ═══════════════════════════════════════
#  FULL MODEL COMPARISON
# ═══════════════════════════════════════
with open(os.path.join(MODEL_DIR, 'baseline_metrics.json'), 'r') as f:
    baseline_metrics = json.load(f)
with open(os.path.join(MODEL_DIR, 'dl_metrics.json'), 'r') as f:
    v1_dl = json.load(f)

all_metrics = {
    'Persistence':   baseline_metrics['Persistence'],
    'XGBoost':       baseline_metrics['XGBoost'],
    'CNN-BiLSTM V1': v1_dl.get('CNN-BiLSTM-Attn', {'val': v1_dl.get('val',{}), 'test': v1_dl.get('test',{})}),
    'CNN-BiLSTM V2': {'val': dl_v2_val, 'test': dl_v2_test}
}

print(f"\n{'='*75}")
print(f"   FULL MODEL COMPARISON (including V2)")
print(f"{'='*75}")
for split_name in ['val', 'test']:
    print(f"\n--- {split_name.upper()} SET ---")
    print(f"{'Model':<20} {'MAE':>8} {'RMSE':>8} {'MAPE%':>8} {'PeakMAE':>10}")
    print("-"*60)
    for model_name, m in all_metrics.items():
        if split_name in m and m[split_name]:
            t = m[split_name]
            print(f"{model_name:<20} {t['MAE']:>8} {t['RMSE']:>8} {t['MAPE']:>8} {t['Peak_MAE']:>10}")

fig, axes = plt.subplots(1, 4, figsize=(22, 5))
metric_names = ['MAE', 'RMSE', 'MAPE', 'Peak_MAE']
model_names  = [m for m in all_metrics if 'test' in all_metrics[m] and all_metrics[m]['test']]
colors = ['#e74c3c', '#3498db', '#95a5a6', '#2ecc71']

for ax, metric in zip(axes, metric_names):
    vals = [all_metrics[m]['test'][metric] for m in model_names]
    bars = ax.bar(model_names, vals, color=colors[:len(model_names)], edgecolor='white', linewidth=1.5)
    ax.set_title(metric, fontsize=13, fontweight='bold'); ax.set_ylabel(metric)
    for bar, v in zip(bars, vals):
        ax.text(bar.get_x()+bar.get_width()/2, bar.get_height()+0.5, f'{v}', ha='center', fontsize=9)
    ax.grid(True, alpha=0.2, axis='y'); ax.tick_params(axis='x', rotation=15)
plt.suptitle('Model Comparison — Test Set (V1 vs V2)', fontsize=15, fontweight='bold', y=1.03)
plt.tight_layout()
plt.savefig(os.path.join(V2_DIR, 'model_comparison_v2.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Improvement over XGBoost
xgb_test = baseline_metrics['XGBoost']['test']
print(f"\n{'='*50}")
print(f"   V2 IMPROVEMENT vs XGBoost")
print(f"{'='*50}")
for metric in ['MAE', 'RMSE', 'MAPE', 'Peak_MAE']:
    xgb_v = xgb_test[metric]
    v2_v  = dl_v2_test[metric]
    pct = (xgb_v - v2_v) / xgb_v * 100
    arrow = '✅↓' if pct > 0 else '❌↑'
    print(f"  {metric:10s}: XGBoost={xgb_v:>8} → V2={v2_v:>8}  ({pct:+.1f}%) {arrow}")

In [ ]:
# Save everything
dl_v2_full = {
    'val': dl_v2_val, 'test': dl_v2_test,
    'horizon_wise': {str(k): v for k, v in horizon_metrics.items()},
    'rolling_origin': {'per_origin': rolling_metrics, 'average': avg_rolling},
    'hyperparameters': HP,
    'total_params': int(total_params),
    'best_val_loss': float(best_val_loss)
}
with open(os.path.join(V2_DIR, 'dl_metrics_v2.json'), 'w') as f:
    json.dump(dl_v2_full, f, indent=2, cls=NumpyEncoder)

import pandas as pd
pd.DataFrame(history).to_csv(os.path.join(V2_DIR, 'dl_training_history_v2.csv'), index=False)

comparison_save = {k: v for k, v in all_metrics.items() if 'test' in v and v['test']}
with open(os.path.join(V2_DIR, 'all_model_comparison_v2.json'), 'w') as f:
    json.dump(comparison_save, f, indent=2, cls=NumpyEncoder)

print("\n✅ All V2 artifacts saved to models/v2/")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
n_test = len(y_pred_test)
for idx, ax in enumerate(axes.flat):
    si = idx * (n_test // 4)
    ax.plot(inverse_scale(y_test_w[si]), 'k-', lw=2, label='Actual')
    ax.plot(inverse_scale(y_pred_test[si]), 'g--', lw=1.5, label='V2 Prediction')
    ax.set_title(f'Test Window #{si}'); ax.set_xlabel('Hour'); ax.set_ylabel('Load (MW)')
    ax.legend(); ax.grid(True, alpha=0.3)
plt.suptitle('CNN-BiLSTM-Attention V2 — Predictions vs Actual', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(V2_DIR, 'dl_sample_predictions_v2.png'), dpi=150, bbox_inches='tight')
plt.show()
print("🎉 V2 complete!")